In [32]:
import fitz
import itertools


def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = itertools.tee(iterable)
    next(b, None)
    return zip(a, b)

doc = fitz.open('../Dossier_Golem/19-021 - LODI_DAE_Volume 3_indice 29_220301.pdf.pdf')

In [46]:
toc = [{'level': item[0], 'title': item[1], **item[3]} for item in doc.get_toc(simple=False)]
for item in toc[:10]: print(item)

{'level': 1, 'title': 'PARTIE 1 : RESUME NON TECHNIQUE DE L’ETUDE D’IMPACT', 'kind': 1, 'xref': 24758, 'page': 12, 'to': Point(65.0, 67.0), 'zoom': 0.0, 'collapse': True}
{'level': 2, 'title': '1 Introduction', 'kind': 1, 'xref': 25306, 'page': 12, 'to': Point(65.0, 99.0), 'zoom': 0.0}
{'level': 2, 'title': '2 Présentation du document', 'kind': 1, 'xref': 25327, 'page': 12, 'to': Point(65.0, 243.0), 'zoom': 0.0}
{'level': 2, 'title': '3 Présentation du projet', 'kind': 1, 'xref': 25323, 'page': 12, 'to': Point(65.0, 348.0), 'zoom': 0.0, 'collapse': True}
{'level': 3, 'title': 'LODI', 'kind': 1, 'xref': 25325, 'page': 12, 'to': Point(94.0, 373.0), 'zoom': 0.0}
{'level': 3, 'title': 'Le projet', 'kind': 1, 'xref': 25330, 'page': 13, 'to': Point(94.0, 67.0), 'zoom': 0.0}
{'level': 3, 'title': 'Les raisons du projet', 'kind': 1, 'xref': 25326, 'page': 13, 'to': Point(94.0, 182.0), 'zoom': 0.0}
{'level': 2, 'title': '4 Impacts du projet et mesures associées', 'kind': 1, 'xref': 25319, 'page

In [16]:
page = doc.load_page(12)
page.bound()

Rect(0.0, 0.0, 595.3200073242188, 842.0399780273438)

In [30]:
blocks = page.get_text('blocks')
sorted(blocks, key=lambda x: x[1])

[(119.9000015258789,
  21.69000244140625,
  479.6146545410156,
  30.69000244140625,
  'Projet de classement SEVESO seuil bas du site de production et stockage de produits anti-nuisibles  \n',
  0,
  0),
 (246.52999877929688,
  32.72998046875,
  350.8246765136719,
  41.72998046875,
  'LODI à Grand-Fougeray (35) \n',
  1,
  0),
 (165.3800048828125,
  43.64996337890625,
  432.0646667480469,
  52.64996337890625,
  '___________________________________________________________ \n',
  2,
  0),
 (65.16000366210938,
  54.69000244140625,
  67.19467163085938,
  63.69000244140625,
  ' \n',
  3,
  0),
 (65.16000366210938,
  67.10995483398438,
  464.358154296875,
  83.06995391845703,
  'PARTIE 1 : RESUME NON TECHNIQUE DE L’ETUDE D’IMPACT \n',
  9,
  0),
 (65.16000366210938,
  93.46277618408203,
  177.2681427001953,
  115.439697265625,
  '1 Introduction  \n',
  10,
  0),
 (65.16000366210938,
  123.01998901367188,
  532.6849975585938,
  162.0199432373047,
  ' \nLe résumé non technique répond aux prescr

In [89]:
# FIXME: two columns

def get_text_blocks(doc, title, next_title):
    for page_no in range(title['page'], next_title.get('page', doc.page_count - 1) + 1):
        page = doc.load_page(page_no)
        for x0, y0, x1, y1, text, block_no, block_type in sorted(page.get_text('blocks'), key=lambda x: x[1]):
            if block_type != 0 or (page_no == title['page'] and y1 < title['to'].y):
                continue
            if next_title and page_no == next_title['page'] and y1 >= next_title['to'].y:
                return
            yield x0, y0, x1, y1, text
    

# add empty title for last one
level = []
for ititle, (title, next_title) in enumerate(pairwise(toc + [{}])):
    text_blocks = list(get_text_blocks(doc, title, next_title))
    toc[ititle]['text'] = text_blocks
    level = [j for i, j in enumerate(level) if i < title['level'] - 1] + [title['title']]
    toc[ititle]['all_levels'] = level


In [90]:
toc

[{'level': 1,
  'title': 'PARTIE 1 : RESUME NON TECHNIQUE DE L’ETUDE D’IMPACT',
  'kind': 1,
  'xref': 24758,
  'page': 12,
  'to': Point(65.0, 67.0),
  'zoom': 0.0,
  'collapse': True,
  'text': [(65.16000366210938,
    67.10995483398438,
    464.358154296875,
    83.06995391845703,
    'PARTIE 1 : RESUME NON TECHNIQUE DE L’ETUDE D’IMPACT \n')],
  'all_levels': ['PARTIE 1 : RESUME NON TECHNIQUE DE L’ETUDE D’IMPACT']},
 {'level': 2,
  'title': '1 Introduction',
  'kind': 1,
  'xref': 25306,
  'page': 12,
  'to': Point(65.0, 99.0),
  'zoom': 0.0,
  'text': [(65.16000366210938,
    93.46277618408203,
    177.2681427001953,
    115.439697265625,
    '1 Introduction  \n'),
   (65.16000366210938,
    123.01998901367188,
    532.6849975585938,
    162.0199432373047,
    ' \nLe résumé non technique répond aux prescriptions des articles R.122-5 (II-1°) du Code de \nl’Environnement. \n'),
   (65.16000366210938,
    174.85995483398438,
    532.4727172851562,
    227.7799530029297,
    'Le projet

In [97]:
for item in toc:
    for level, title in enumerate(item['all_levels']):
        print('\t'*level + '*** ' + title)
    print('\n'.join(i[-1] for i in item['text']))
    print('_.'*60)

*** PARTIE 1 : RESUME NON TECHNIQUE DE L’ETUDE D’IMPACT
PARTIE 1 : RESUME NON TECHNIQUE DE L’ETUDE D’IMPACT 

_._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._.
*** PARTIE 1 : RESUME NON TECHNIQUE DE L’ETUDE D’IMPACT
	*** 1 Introduction
1 Introduction  

 
Le résumé non technique répond aux prescriptions des articles R.122-5 (II-1°) du Code de 
l’Environnement. 

Le projet est soumis à étude d’impact systématique, étant précisé que la présente étude d’impact fera 
l’objet, à l’occasion de l’instruction de la demande d’autorisation, d’une enquête publique, au titre de 
la rubrique 1.b de la nomenclature annexée à l’article R122-2 du code de l’environnement. 
 

_._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._.
*** PARTIE 1 : RESUME NON TECHNIQUE DE L’ETUDE D’IMPACT
	*** 2 Présentation du document
2 Présentation du document 

 
Le résumé non techniq

In [91]:
for item in toc:
    print(item['all_levels'])
    # print('_.'*60)

['PARTIE 1 : RESUME NON TECHNIQUE DE L’ETUDE D’IMPACT']
['PARTIE 1 : RESUME NON TECHNIQUE DE L’ETUDE D’IMPACT', '1 Introduction']
['PARTIE 1 : RESUME NON TECHNIQUE DE L’ETUDE D’IMPACT', '2 Présentation du document']
['PARTIE 1 : RESUME NON TECHNIQUE DE L’ETUDE D’IMPACT', '3 Présentation du projet']
['PARTIE 1 : RESUME NON TECHNIQUE DE L’ETUDE D’IMPACT', '3 Présentation du projet', 'LODI']
['PARTIE 1 : RESUME NON TECHNIQUE DE L’ETUDE D’IMPACT', '3 Présentation du projet', 'Le projet']
['PARTIE 1 : RESUME NON TECHNIQUE DE L’ETUDE D’IMPACT', '3 Présentation du projet', 'Les raisons du projet']
['PARTIE 1 : RESUME NON TECHNIQUE DE L’ETUDE D’IMPACT', '4 Impacts du projet et mesures associées']
['PARTIE 1 : RESUME NON TECHNIQUE DE L’ETUDE D’IMPACT', '4 Impacts du projet et mesures associées', 'Milieu physique']
['PARTIE 1 : RESUME NON TECHNIQUE DE L’ETUDE D’IMPACT', '4 Impacts du projet et mesures associées', 'Milieu physique', '4.1.1 Localisation de la zone d’étude']
['PARTIE 1 : RESUME NON

In [72]:
# Remove repeated blocks (header and footer)
block_count = {}
for item in toc:
    for text_block in item['text']:
        key = text_block[:-1]
        block_count[key] = (block_count.get(key, [0])[0] + 1, text_block[-1])

In [78]:
for pos, (count, text) in sorted(block_count.items(), key=lambda x: x[1][0]):
    if count > 5:
        print(count, pos, text, sep='\n')

6
(82.31999969482422, 84.28279876708984, 188.7541961669922, 103.75773620605469)
Article 
Ventilation 

6
(146.89999389648438, 84.28279876708984, 570.162353515625, 109.51774597167969)
Stock 
Désignation 
zone future A MP : local des 

6
(452.4700012207031, 90.04280853271484, 719.1488647460938, 109.51774597167969)
matières techniques 
Type de produits 

7
(402.04998779296875, 37.28997802734375, 790.504638671875, 46.28997802734375)
Projet d’extension du site de production et stockage de produits anti-nuisibles LODI à Grand-Fougeray (35) 

7
(462.04998779296875, 48.2099609375, 730.6246948242188, 57.2099609375)
 ___________________________________________________________ 

7
(324.3800048828125, 83.3227767944336, 368.6622314453125, 97.03770446777344)
Volatilité 

7
(890.1099853515625, 83.3227767944336, 940.4976806640625, 108.55772399902344)
Potentiel 
de danger 

7
(259.4599914550781, 94.8427963256836, 314.87750244140625, 120.07768249511719)
Tension de 
vapeur (Pa 

7
(321.5, 94.842796325683

In [93]:
from IPython.display import Markdown as md
md("""
- level 1
  - level 2
    - level 3
""")
       


- level 1
  - level 2
    - level 3
